In [1]:
# To be able to use the quantools, due to my crap path names have to add to sys path
import sys
sys.path.insert(0, '/home/adam/Dropbox/2-creations/2-crafts/7-buidl/0-utils/quant_tools/code')

# IMPORT PACKAGES
from tensorflow.keras import layers, regularizers
from tensorflow.keras.layers import Layer
from keras.callbacks import EarlyStopping
from typing import List, Optional, Tuple, Dict
from tools import QuantTools
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import itertools
import pickle
import time
import gc

keras.mixed_precision.set_global_policy("mixed_float16")


2023-07-27 14:28:03.155938: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-27 14:28:03.207365: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 14:28:04.067560: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070 Ti, compute capability 8.6


In [2]:
def subsetRowsAndColumns(
    df: pd.DataFrame, lhs_col: str
    ) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    ''' Subset relevant rows and columns to form needed dataframes.

    Parameters:
        df      (pd.DataFrame): The original DataFrame containing the data.
        lhs_col (str):          The name of lhs column.

    Notes:
        - Kept characteristics based on what are most raw characteristics and 
            what work well as univariate factor to gen large spread in returns
            in pre 2h 2022 data.
        - Kept matching number of macro cols up to integer scalar which have
            high corr to avg LHS after taking out variabilitity of previous
            chosen columns.
        - Keep also previous returns as a separate dataframe for the factor
            side of the network.

    Returns:
        tuple: A tuple containing three DataFrames:
            y_df: A DataFrame with relevant rows and lhs column.
            char_df: A DataFrame with raw characteristics.
            ts_df: A DataFrame containing previous returns.
    '''
    # Set column lists that I manually constructed
    char_cols_to_keep = ['char_addr_active_tm1h',
        'char_addr_new_tm1h',
        'char_alpha_tm7',
        'char_beta_tm7',
        'char_bidask_t',
        'char_delta_flow_dist_tm1h',
        'char_delta_holders_dist_tm1h',
        'char_exchange_inflow_tm1h',
        'char_exchange_outflow_tm1h',
        'char_ivol_tm30',
        'char_prct_supply_in_profit_t',
        'char_r_ath_t',
        'char_r_atl_t',
        'char_r_tm1h',
        "char_r_tm14",
        'char_sent_neg_twitter_tm1',
        'char_sent_volume_consumed_tm1',
        'char_size_t',
        'char_social_volume_tm1',
        'char_trades_t',
        'char_turnover_tm1',
        'char_tx_volume_t',
        'char_var5_tm1',
        "char_vol_tm6h"]
    macro_cols_to_keep = ['macro_avg_fee_med_usd_t',
                        'macro_btc_adr_act_cnt_t',
                        'macro_btc_cap_act_1yr_usd_t',
                        'macro_btc_fee_med_usd_t',
                        'macro_btc_flow_in_ex_usd_t',
                        'macro_btc_flow_out_ex_usd_t',
                        'macro_btc_mvrv_t',
                        'macro_btc_sply_act_30d_t',
                        'macro_btc_sply_utxo_loss_t',
                        'macro_btc_sply_utxo_prof_t',
                        'macro_btc_tx_tfr_val_med_usd_t',
                        'macro_btc_utxo_age_med_t',
                        'macro_cmkt_tm1h',
                        'macro_cp3mx_t',
                        'macro_cpiaucsl_t',
                        'macro_emv_inflation_t',
                        'macro_emv_t',
                        'macro_eth_adr_act_cnt_t',
                        'macro_eth_cap_act_1yr_usd_t',
                        'macro_eth_fee_med_t',
                        'macro_eth_flow_in_ex_usd_t',
                        'macro_eth_flow_out_ex_usd_t',
                        'macro_eth_roi_t',
                        'macro_eth_stakers_count_t',
                        'macro_eth_tx_tfr_val_med_usd_t',
                        'macro_eth_vel_cur_1yr_t',
                        'macro_ex_open_interest_future_usd_t',
                        'macro_ex_usd_volume_24h_cex_t',
                        'macro_ex_volume_future_usd_t',
                        'macro_ex_volume_t',
                        'macro_expinf1yr_t',
                        'macro_fedfunds_t',
                        'macro_gs1_t',
                        'macro_invest_t',
                        'macro_m2sl_t',
                        'macro_mvrv_med_t',
                        'macro_oilpricex_t',
                        'macro_snp500_t',
                        'macro_snp_pe_t',
                        'macro_stablecoin_dev_t',
                        'macro_t10yie_t',
                        'macro_tmu_sca_t',
                        'macro_total_adr_bal_cnt_t',
                        'macro_total_cap_act_1yr_usd_t',
                        'macro_total_maker_deposits_t',
                        'macro_total_tx_tfr_val_adj_usd_t',
                        'macro_us_mpu_t',
                        'macro_vixclsx_t']

    # Dropping pre 2020 rows so when we create a balanced panel we have more real data than fake
    df = df[df.date >= '2020-01-01'].copy()

    # Form dataframe of all dates and assets and merge back on df to ensure obs for all assets
    unique_dates  = df.date.unique()
    unique_assets = df.asset.unique()
    cross_product = list(itertools.product(unique_dates, unique_assets))
    cross_df      = pd.DataFrame(cross_product, columns=['date', 'asset'])
    df            = df.merge(
        cross_df, on=['date', 'asset'], how='outer', validate='one_to_one')
    df            = df.sort_values(by=['date', 'asset'], ignore_index=True)
    assert(len(df) == (len(np.unique(df.date))*len(np.unique(df.asset))))

    # Form dataframe
    y_df = df[['date', 'asset', lhs_col]].copy()
    y_df = y_df.sort_values(by=['date', 'asset'], ignore_index=True)

    # Form characteristic dataframe
    char_df = df[['date', 'asset']+char_cols_to_keep].copy()
    char_df = char_df.sort_values(by=['date', 'asset'], ignore_index=True)

    # Form previous return dataframe by extracting from char_df and reshaping wide
    r_df         = char_df[['date', 'asset', 'char_r_tm1h']].copy()
    r_df         = r_df.pivot(index='date', columns='asset', values='char_r_tm1h').reset_index()
    r_df.columns = (['date'] 
        + ['asset_r_' + str(col) + '_t1h' for col in r_df.columns if col != 'date'])

    # Form ts_df as the macro columns plus the previous return columns
    macro_df = df[df.asset=='btc'][['date']+macro_cols_to_keep].copy()
    assert(set(macro_df.date)==set(r_df.date))
    ts_df    = macro_df.merge(r_df, on='date', how='inner', validate='one_to_one')
    ts_df    = ts_df.sort_values(by='date', ignore_index=True)

    # Run final checks
    assert(set(y_df.date)==set(char_df.date))
    assert(set(y_df.date)==set(ts_df.date))
    assert(0 == ((len(macro_cols_to_keep) 
                / len(char_cols_to_keep)) % 1)) # macro cols are some multiple of # char cols

    return y_df, char_df, ts_df


In [3]:
def normalizeAndFillMissing(df: pd.DataFrame, 
        lhs_col: Optional[str] = None, lhs_pad: float = -2, rhs_pad: float = -2,
        ignore_cols: Optional[List[str]] = None) -> pd.DataFrame:
    """
    Normalize all numeric columns of a DataFrame to be between -1 and 1 based on rank,
    and set missing values to given lhs and rhs pad values.

    Parameters:
    df          (pd.DataFrame):        Input DataFrame to normalize.
    lhs_col     (str, optional):       Column name to fill missing values with lhs_pad.
    lhs_pad     (float, optional):     Value to replace missing values in lhs_col.
    rhs_pad     (float, optional):     Value to replace missing values in other columns.
    ignore_cols (List[str], optional): List of column names to ignore during normalization.

    Returns:
    pd.DataFrame: Normalized DataFrame.

    Raises:
    ValueError: If any column to normalize is not numeric.
    """
    # If ignore_cols is not provided, use an empty list
    if ignore_cols is None:
        ignore_cols = []

    # Get the columns to normalize
    cols_to_normalize = [col for col in df.columns if col not in ignore_cols]

    # Check if all columns to normalize are numeric
    if not all(dtype.kind in 'biufc' for dtype in df[cols_to_normalize].dtypes):
        raise ValueError("All columns to normalize must be numeric")

    # Check for constant columns and raise a warning
    constant_columns = df[cols_to_normalize].columns[df[cols_to_normalize].nunique() <= 1]
    if len(constant_columns) > 0:
        print(f"Warning: Columns {constant_columns} have constant values and will result in NaN after normalization.")

    # Find out how many missing in the cols to normalize 
    missing_per_column = df[cols_to_normalize].isnull().sum()

    # Rank the values in each column 
    #    -where we first rank normalize
    #    -but then add noise and 
    #         re rank normalize to ensure unique values with 
    #         even coverage of RHS latent space
    if len(cols_to_normalize) > 0:
        df[cols_to_normalize] = 2*((df[cols_to_normalize].rank() 
                                    / (len(df) - missing_per_column)) - 0.5)
        df[cols_to_normalize] = (df[cols_to_normalize] 
            + np.random.uniform(-1e-6, 1e-6, (len(df), len(cols_to_normalize))))
        df[cols_to_normalize] = (2*((df[cols_to_normalize].rank() 
                                    / (len(df) - missing_per_column)) - 0.5))

    # Replace missing LHS and RHS with given buffer values
    if lhs_col is not None and lhs_col in df.columns:
        df[lhs_col].fillna(lhs_pad, inplace=True)
        assert np.allclose(0, df[lhs_col].isnull().sum())
    df.fillna(rhs_pad, inplace=True)
    assert 0 == df.isnull().sum().sum()

    # Compress data, resort, and reset index
    for col in df.select_dtypes(include=[np.float64]).columns:
        df[col] = df[col].astype(np.float32)
    if 'asset' in df.columns:
        df      = df.sort_values(by=['date', 'asset'], ignore_index=True)
    else:
        df      = df.sort_values(by=['date'], ignore_index=True)

    return df


In [4]:
def formLhsAndRhsTensors(rel_y_df: pd.DataFrame, rel_char_df: pd.DataFrame, rel_ts_df: pd.DataFrame,
                            datetimes_window: List[pd.Timestamp], prev_asset_ret_cols: List[str],
                            macro_cols: List[str], lhs_col: str, num_lags: int, num_assets: int,
                            num_chars: int) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    """
    Form tensors for loading input, factor input, and output for machine learning model.

    Parameters:
        rel_y_df            (pd.DataFrame):       DataFrame containing target variable (response) data.
        rel_char_df         (pd.DataFrame):       DataFrame containing characteristics data.
        rel_ts_df           (pd.DataFrame):       DataFrame containing time series data (factors).
        datetimes_window    (List[pd.Timestamp]): List of datetime values for which to form tensors.
        prev_asset_ret_cols (List[str]):          List of column names for previous asset returns in `rel_ts_df`.
        macro_cols          (List[str]):          List of column names for macroeconomic data in `rel_ts_df`.
        lhs_col             (str):                The column name in `rel_y_df` representing the target variable.
        num_lags            (int):                Number of lagged time steps to consider.
        num_assets          (int):                Number of assets (entities).
        num_chars           (int):                Number of characteristics.

    Returns:
        Tuple[tf.Tensor, tf.Tensor, tf.Tensor]: A tuple containing three tensors:
            - `loading_input`: Tensor containing the concatenated characteristics and macro data.
            - `factor_input`: Tensor containing previous asset return data.
            - `output`: Tensor containing the target returns.

    Raises:
        ValueError: If the input data shapes are not as expected.
    """
    # Initialize lists to return
    loading_input_list = []
    factor_input_list  = []
    output_list        = []

    # Loop over all datetimes to form LHS and RHS for
    for i, datetime in enumerate(datetimes_window):
        # Form the beginning datetime for this observation given the number of lags to step back
        datetime_input_start = datetime - pd.Timedelta(hours=num_lags - 1)

        # Obtain output data
        output_list.append(rel_y_df.loc[rel_y_df.date == datetime, lhs_col].values)

        # Obtain input data
        rel_char_filtered = rel_char_df[
            (rel_char_df.date >= datetime_input_start) & (rel_char_df.date <= datetime)
            ].drop(columns=['date', 'asset'])

        char_data = rel_char_filtered.values.reshape(
                        (num_lags, num_assets, num_chars)
                        ).transpose((0, 2, 1))

        rel_ts_filtered = rel_ts_df[
            (rel_ts_df.date >= datetime_input_start) & (rel_ts_df.date <= datetime)]

        if len(rel_ts_filtered) < num_lags:
            raise ValueError("Not enough data points for the given number of lags.")

        macro_data = rel_ts_filtered[macro_cols].values.reshape((num_lags, num_chars, -1))

        # Append to lists of input data
        loading_input_list.append(np.concatenate((char_data, macro_data), axis=2))
        factor_input_list.append(rel_ts_filtered[prev_asset_ret_cols].values)

    # Convert validation data lists to tensors
    loading_input = tf.convert_to_tensor(np.array(loading_input_list), dtype=tf.float32)
    factor_input  = tf.convert_to_tensor(np.array(factor_input_list), dtype=tf.float32)
    output        = tf.convert_to_tensor(np.array(output_list), dtype=tf.float32)

    return loading_input, factor_input, output


In [5]:
class TransformerEncoder(layers.Layer):
    def __init__(self, num_heads: int, hidden_dim: int, dropout_pct: float,
                 dense_dim: int, l2_penalty: float, **kwargs):
        """
        TransformerEncoder layer.

        Parameters:
            num_heads   (int):   Number of attention heads in the multi-head attention layer.
            hidden_dim  (int):   Dimension of the hidden layers.
            dropout_pct (float): Dropout rate as a percentage.
            dense_dim   (int):   Dimension of the dense layers.
            l2_penalty  (float): L2 regularization penalty.
            **kwargs: Additional arguments for the base Layer class.
        """
        super().__init__(**kwargs)
        self.num_heads   = num_heads
        self.hidden_dim  = hidden_dim
        self.dropout_pct = dropout_pct
        self.dense_dim   = dense_dim
        self.l2_penalty  = l2_penalty
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=hidden_dim,
            kernel_initializer='glorot_uniform',
            bias_initializer='random_uniform',
            dropout = dropout_pct,
            kernel_regularizer=regularizers.l2(l2=l2_penalty))
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation='gelu',
                        kernel_initializer='glorot_uniform',
                        bias_initializer='random_uniform',
                        kernel_regularizer=regularizers.l2(l2=l2_penalty)),
                layers.Dropout(dropout_pct),
                layers.Dense(hidden_dim, activation='linear',
                            kernel_initializer='glorot_uniform',
                            bias_initializer='random_uniform',
                            kernel_regularizer=regularizers.l2(l2=l2_penalty)),
                layers.Dropout(dropout_pct),
                ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
    
    def call(self, inputs):
        attention_output = self.attention(query=inputs, key=inputs, value=inputs)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "num_heads":  self.num_heads,
            "hidden_dim": self.hidden_dim,
            "dropout_pct": self.dropout_pct,
            "dense_dim":  self.dense_dim,
            "l2_penalty": self.l2_penalty,
        })
        return config

def buildLoadingOutputs(inputs: Layer, num_assets: int, hidden_dim: int, l2_penalty: float,
                        dropout_pct: float, num_heads: int, dense_dim: int, num_factors: int) -> Layer:
    """
    Build loading outputs for the Transformer model.

    Parameters:
        inputs      (Layer): Input layer for the encoder.
        num_assets  (int):   Number of assets (entities).
        hidden_dim  (int):   Dimension of the hidden layers.
        l2_penalty  (float): L2 regularization penalty.
        dropout_pct (float): Dropout rate as a percentage.
        num_heads   (int):   Number of attention heads.
        dense_dim   (int):   Dimension of the dense layers.
        num_factors (int):   Number of factors in the model.

    Returns:
        Layer: Output layer of the loading outputs.
    """

    encoder_outputs = []
    for _ in range(num_assets):
        encoder_output = layers.Dense(hidden_dim, activation='linear',
                                    kernel_initializer='glorot_uniform',
                                    bias_initializer='random_uniform',
                                    kernel_regularizer=regularizers.l2(l2=l2_penalty))(inputs)
        encoder_output = layers.Dropout(dropout_pct)(encoder_output)
        encoder_output = TransformerEncoder(num_heads, hidden_dim, dropout_pct, dense_dim, l2_penalty)(encoder_output)
        encoder_output = layers.GlobalAveragePooling2D()(encoder_output)
        encoder_output = layers.Dense(num_factors, activation='linear',
                                    kernel_initializer='glorot_uniform',
                                    bias_initializer='random_uniform',
                                    kernel_regularizer=regularizers.l2(l2=l2_penalty))(encoder_output)
        encoder_outputs.append(encoder_output)

    # Stack the outputs and reshape to a matrix of dim num_assets by num_factors
    outputs = layers.Concatenate(axis=1)(encoder_outputs)  
    output  = layers.Reshape((num_assets, num_factors))(outputs)
    
    return output

def buildFactorOutputs(inputs: Layer, hidden_dim: int, l2_penalty: float, dropout_pct: float,
                        num_heads: int, dense_dim: int, num_factors: int) -> Layer:
    """
    Build factor outputs for the Transformer model.

    Parameters:
        inputs      (Layer): Input layer for the encoder.
        hidden_dim  (int):   Dimension of the hidden layers.
        l2_penalty  (float): L2 regularization penalty.
        dropout_pct (float): Dropout rate as a percentage.
        num_heads   (int):   Number of attention heads.
        dense_dim   (int):   Dimension of the dense layers.
        num_factors (int):   Number of factors in the model.

    Returns:
        Layer: Output layer of the factor outputs.
    """
    encoder_output = layers.Dense(hidden_dim, activation='linear',
                                kernel_initializer='glorot_uniform',
                                bias_initializer='random_uniform',
                                kernel_regularizer=regularizers.l2(l2=l2_penalty))(inputs)
    encoder_output = layers.Dropout(dropout_pct)(encoder_output)
    encoder_output = TransformerEncoder(num_heads, hidden_dim, dropout_pct, dense_dim, l2_penalty)(encoder_output)
    encoder_output = layers.GlobalAveragePooling1D()(encoder_output)
    output = layers.Dense(num_factors, activation='linear',
                        kernel_initializer='glorot_uniform',
                        bias_initializer='random_uniform')(encoder_output)
    return output

def buildTransformer(num_chars: int, num_macro_vectors: int, 
    num_assets: int, num_training_obs: int, 
    hps_dict: Dict) -> keras.Model:
    """
    Build and compile a Transformer model.

    Parameters:
        num_chars (int): Number of characteristics.
        num_macro_vectors (int): Number of macroeconomic vectors.
        num_assets (int): Number of assets (entities).
        num_training_obs (ints): Number of observations in the training data for this model.
        hps_dict (Dict): hyperparameter values.

    Returns:
        keras.Model: Compiled Transformer model.
    """
    # Set dense dim to be two times that of hidden dimension
    dense_dim = int(hps_dict['hidden_dim']*2)
    
    # Build inputs
    input_loadings = keras.Input(shape=(hps_dict['num_lags'], num_chars, num_assets + num_macro_vectors))
    input_factors = keras.Input(shape=(hps_dict['num_lags'], num_assets))

    # Build mappings to outputs
    output_loadings = buildLoadingOutputs(
        input_loadings, num_assets, hps_dict['hidden_dim'],
        hps_dict['l2_penalty'], hps_dict['dropout_pct'], 
        hps_dict['num_heads'], dense_dim, hps_dict['number_factors'])
    output_factors = buildFactorOutputs(
        input_factors, hps_dict['hidden_dim'],
        hps_dict['l2_penalty'], hps_dict['dropout_pct'], 
        hps_dict['num_heads'], dense_dim, hps_dict['number_factors'])
    output = layers.Dot(axes=[2, 1])([output_loadings, output_factors])

    # Build optimizer
    decay_steps = int(num_training_obs / hps_dict['batch_size'])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        hps_dict['initial_learning_rate'],
        decay_steps=decay_steps,
        decay_rate=hps_dict['learning_decay_rate'],
        staircase=False)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule,
        beta_1=hps_dict['adam_beta_1'], beta_2=hps_dict['adam_beta_2'], 
        clipnorm=hps_dict['adam_clipnorm'])

    # Build and compile model
    model = keras.Model(inputs=[input_loadings, input_factors], outputs=output)
    model.compile(optimizer=optimizer, 
        loss='mean_squared_error',
        metrics=['mse'], 
        weighted_metrics=['mse'])

    return model


In [6]:
def fitTransformer(model: keras.Model,
        train_loading_input: tf.Tensor, train_factor_input: tf.Tensor, train_output: tf.Tensor, 
        rel_y_df: pd.DataFrame, lhs_col: str, lhs_pad: int, tc_per_hour: float, num_prtfl_qntls: int,
        train_datetimes: List[np.datetime64], num_assets: int, 
        hps_dict: Dict,
        val_loading_input: tf.Tensor = None, val_factor_input: tf.Tensor = None, val_output: tf.Tensor = None, 
    ) -> Tuple[keras.Model, int, float, float]:
    """
    Fit the Transformer model.

    Parameters: 
        model (keras.Model): The Transformer model to be trained.
        train_loading_input (tf.Tensor): Training loading input tensor.
        train_factor_input (tf.Tensor): Training factor input tensor.
        train_output (tf.Tensor): Training output tensor.
        rel_y_df (pd.DataFrame):
        lhs_col (str): Name of LHS column.
        lhs_pad (int): The padded value for the lhs to signify missing.
        tc_per_hour (float): Transaction costs per hour in simple return.
        num_prtfl_qntls (int): Number of quantiles for long-short portfolio construction.
        train_datetimes (List[np.datetime64]): date at which validation period begins.
        num_assets (int): Number of assets (entities).
        hps_dict (Dict): hyperparameter values.
        val_loading_input (tf.Tensor, optional): Validation loading input tensor.
        val_factor_input (tf.Tensor, optional): Validation factor input tensor.
        val_output (tf.Tensor, optional): Validation output tensor.
        
    Returns:
        Tuple: A tuple containing the trained model, number of epochs trained,
                training r^2 predictive, and training geometric average return.
    """
    # Build sample weights to down weight hours where lots of missing asset-hours
    # NOTE: this will downweight past obs relevant to recent obs
    sample_weight_mat = tf.cast(train_output != lhs_pad, dtype=tf.int16)
    sample_weight_array = np.sum(sample_weight_mat, axis=1) / num_assets

    # Fit the model
    callbacks = []
    if hps_dict['early_stopping']:
        es = EarlyStopping(monitor='val_mse', mode='min', verbose=1, patience=5)
        callbacks.append(es)

    if val_loading_input is not None and val_factor_input is not None and val_output is not None:
        validation_data = ([val_loading_input, val_factor_input], val_output)
    else:
        validation_data = None

    model.fit(x=[train_loading_input, train_factor_input],
                y=train_output,
                batch_size=hps_dict['batch_size'], epochs=hps_dict['num_epochs'], verbose=1, callbacks=callbacks,
                validation_data=validation_data,
                sample_weight=sample_weight_array)

    num_epochs_trained = es.stopped_epoch if hps_dict['early_stopping'] else hps_dict['num_epochs']

    # Predict on the training data to return the training r^2_pred for obs with nonmissing return
    train_yhats = model.predict([train_loading_input, train_factor_input])
    train_mask = train_output != lhs_pad
    train_r2_pred = (1 
        - np.mean(np.square(train_output[train_mask] - train_yhats[train_mask])) 
            / np.mean(np.square(train_output[train_mask] - 0)))

    # Form array of training yhats
    train_yhats_array = tf.reshape(train_yhats, [-1]).numpy()

    # Form DataFrame of y and yhat values for nonmissing returns in training window
    train_pos_df = rel_y_df[rel_y_df.date.isin(train_datetimes)].copy()
    train_pos_df['yhats'] = train_yhats_array
    train_pos_df = train_pos_df[train_pos_df[lhs_col] != lhs_pad].reset_index(drop=True)

    # Calculate the geom avg return of given quantile long short portfolios
    train_pos_df = QuantTools.formPortfolioPositionsQuantileLongShort(
            train_pos_df, num_prtfl_qntls)
    train_pos_df['returns'] = train_pos_df.position*train_pos_df[lhs_col]
    returns = train_pos_df.groupby('date')['returns'].sum()
    train_geom_mean_rtrn = QuantTools.calcGeomAvg(returns-tc_per_hour)
    
    return model, num_epochs_trained, train_r2_pred, train_geom_mean_rtrn


In [7]:
def runCV(y_df: pd.DataFrame, char_df: pd.DataFrame, ts_df: pd.DataFrame, 
    asset_universe_dict: Dict[str, List],
    val_start_date: str, val_end_date: str, test_start_date: str, lhs_col: str,
    lhs_pad: int, rhs_pad: int, num_prtfl_qntls: int, tc_per_hour: float,
    hp_grid: Dict[str, list], periods_in_year: int, 
    cv_out_fp: str, arch_name: str) -> List[dict]:
    """
    Run custom step forward cross-validation.

    This function evaluates the performance of the Transformer-based factor model. 
    It uses the input data and hyperparameter grid to train multiple models with 
    different hyperparameter combinations and evaluates their performance using 
    step-forward cross-validation. The function outputs the results to a csv and
    returns a list of dictionaries containing the evaluation results for each model.

    Parameters:
        y_df (pd.DataFrame): DataFrame containing the target variable data.
        char_df (pd.DataFrame): DataFrame containing the characteristic data.
        ts_df (pd.DataFrame): DataFrame containing the previous return data.
        asset_universe_dict (Dict[str, List]): Dictionary containing the asset universe 
                                                for each month in the study period.
        val_start_date (str): Start date of the validation period in 'YYYY-MM-DD' format.
        val_end_date (str): End date of the validation period in 'YYYY-MM-DD' format.
        test_start_date (str): Start date of the test period in 'YYYY-MM-DD' format.
        lhs_col (str): The name of the target variable (lhs) column in y_df.
        lhs_pad (int): The value to pad missing lhs values with.
        rhs_pad (int): The value to pad missing rhs values with.
        num_prtfl_qntls (int): Number of quantiles for long-short portfolio construction.
        tc_per_hour (float): Transaction cost per hour for calculating returns.
        hp_grid (Dict[str, list]): Hyperparameter grid to search for the best model.
        periods_in_year (int): Number of periods in a year for annualization.
        cv_out_fp (str): Filepath to save the cross-validation results in CSV format.
        arch_name (str): Name of the architecture/model being tested.

    Returns:
        List[dict]: A list of dictionaries containing evaluation results for each model 
            in the hyperparameter grid. Each dictionary includes the model's hyperparameters, 
            evaluation metrics, and other relevant information.
    """
    # Subset the LHS and RHS to remove test period
    y_df = y_df[y_df.date < test_start_date].copy()
    char_df = char_df[char_df.date < test_start_date].copy()
    ts_df = ts_df[ts_df.date < test_start_date].copy()

    # Initialize cv result objects
    results_list = []

    # Determine RHS column names
    char_cols = list(char_df.columns.values)
    char_cols.remove('date')
    char_cols.remove('asset')
    macro_cols = [col for col in ts_df.columns if 'macro' in col]
    ret_lag_cols = [col for col in ts_df.columns if 'asset_r_' in col]

    # Determine number of RHS values
    num_chars  = len(char_cols)
    num_macro_vectors = int(len(macro_cols)/len(char_cols))

    # Determine validation datetimes to loop over and datetimes to refit at
    val_dts_dict = {}
    val_datetimes = np.unique(y_df[y_df.date>=val_start_date].date.values)
    val_sun_midnights = np.unique(y_df[(y_df.date>=val_start_date) 
        & (y_df.date.dt.hour==0) & (y_df.date.dt.day_of_week==6)].date.values)

    # Check if first val date is sunday midnight, if not then add the dates
    first_val_date = np.min(y_df[(y_df.date==val_start_date)].date.values)
    day_of_week_of_first_val_datetime = (first_val_date.astype('datetime64[D]').view('int64') - 4) % 7
    if day_of_week_of_first_val_datetime != 6:
        val_dts_dict[first_val_date] = np.unique(y_df[(y_df.date>=first_val_date) & (y_df.date<val_sun_midnights[0])].date.values)

    # Complete the dictionary with all the sundays as keys as the dates until the next sunday as the values
    for val_sun_midnight in val_sun_midnights:
        next_sun_midnight = val_sun_midnight + np.timedelta64(7, 'D')
        val_dts_dict[val_sun_midnight] = np.unique(y_df[(y_df.date>=val_sun_midnight) 
                                            & (y_df.date<next_sun_midnight)
                                            & (y_df.date<test_start_date)].date.values)

    # Loop over hp combinations
    keys = hp_grid.keys()
    values = hp_grid.values()
    hp_combos = list(itertools.product(*values))
    for hps in hp_combos:
        # Start the timer
        tic = time.perf_counter()

        # Create hp dictionary and other objects for this iteration
        hps_dict = dict(zip(keys, hps))
        hps_results_dict = hps_dict.copy()
        val_y_yhats_df = pd.DataFrame()

        # Report on progress
        print(hps_dict)

        # Initiate lists for results and start the loop over the val dates to fit and predict
        num_epochs_trained_list   = []
        train_r2_pred_list        = []
        train_geom_mean_rtrn_list = []
        num_model_params_list     = []
        for val_datetime_start in list(val_dts_dict.keys()): 
            print(val_datetime_start)
            # form training and validation datetime objects
            train_datetimes = list(ts_df[ts_df.date < val_datetime_start].date.values)[hps_dict['num_lags']-1:]
            val_datetimes_window = val_dts_dict[val_datetime_start]
            val_datetime_end = np.max(val_datetimes_window)

            # form appropriate asset universe and update num asset parameter
            first_day_of_month_for_current_val_dt = np.datetime_as_string(val_datetime_start, unit='M')+'-01'
            asset_universe = asset_universe_dict[first_day_of_month_for_current_val_dt]
            num_assets = len(asset_universe)

            # figure out what assets are not included in this asset universe to drop from previous return df
            prev_ret_cols_to_drop = [col for col in ts_df.columns 
                                    if (col != 'date') & ('asset_r_' in col) 
                                    if col.split('_')[2] not in asset_universe]

            # for all dfs, cut down assets and form relevant dataframes of up to end of current val week
            rel_y_df    = y_df[(y_df.asset.isin(asset_universe))
                                & ((y_df.date <= val_datetime_end))].copy()
            rel_char_df = char_df[(char_df.asset.isin(asset_universe))
                                & (char_df.date <= val_datetime_end)].copy()
            rel_ts_df   = ts_df[(ts_df.date <= val_datetime_end)].drop(columns=prev_ret_cols_to_drop, axis=1)

            # form rel prev asset return col names
            prev_asset_ret_cols = [col for col in rel_ts_df.columns if 'asset_r_' in col]

            # normalize rhs data (note: this takes 2-15 min given big df's)
            rel_char_df = normalizeAndFillMissing(rel_char_df, lhs_col,
                            lhs_pad, rhs_pad, ignore_cols=['date', 'asset'])
            rel_ts_df = normalizeAndFillMissing(rel_ts_df, lhs_col, 
                            lhs_pad, rhs_pad, ignore_cols='date')

            # form training and validation data
            train_loading_input, train_factor_input, train_output = formLhsAndRhsTensors(rel_y_df, rel_char_df, rel_ts_df,
                                                                        train_datetimes, prev_asset_ret_cols, macro_cols,
                                                                        lhs_col, hps_dict['num_lags'], num_assets, num_chars)
            val_loading_input, val_factor_input, val_output = formLhsAndRhsTensors(rel_y_df, rel_char_df, rel_ts_df,
                                                                        val_datetimes_window, prev_asset_ret_cols, macro_cols,
                                                                        lhs_col, hps_dict['num_lags'], num_assets, num_chars)

            # Check that no assets with missing returns in the validation data
            assert(0 == np.sum(val_output==lhs_pad))

            # Fit and predict
            num_training_obs = train_output.shape[0]
            model = buildTransformer(
                num_chars, num_macro_vectors, num_assets, num_training_obs, hps_dict)
            model, num_epochs_trained, train_r2_pred, train_geom_mean_rtrn = fitTransformer(
                model, train_loading_input, train_factor_input, train_output,
                rel_y_df, lhs_col, lhs_pad, tc_per_hour, num_prtfl_qntls, 
                train_datetimes, num_assets, hps_dict,
                val_loading_input, val_factor_input, val_output)
            val_yhats = model.predict([val_loading_input, val_factor_input])
            val_yhats_array = tf.reshape(val_yhats, [-1]).numpy()

            # Save this val week returns
            num_epochs_trained_list.append(num_epochs_trained)
            train_r2_pred_list.append(train_r2_pred)
            train_geom_mean_rtrn_list.append(train_geom_mean_rtrn)
            num_model_params_list.append(model.count_params())
            temp_yhats_df = rel_y_df[rel_y_df.date >= val_datetime_start].reset_index(drop=True).copy()
            temp_yhats_df['yhats'] = val_yhats_array
            val_y_yhats_df = pd.concat([val_y_yhats_df, temp_yhats_df])

            # Output this week's results
            if True:
                val_week_df = val_y_yhats_df[(val_y_yhats_df.date>=val_datetime_start) 
                                        & (val_y_yhats_df.date<=val_datetime_end)].copy()
                val_week_df = val_week_df.sort_values(by=['date', 'asset'], ignore_index=True)
                val_week_df = QuantTools.formPortfolioPositionsQuantileLongShort(val_week_df, num_prtfl_qntls)
                val_week_df['returns'] = val_week_df.position*val_week_df[lhs_col]
                val_week_y = val_week_df[lhs_col].values
                val_week_yhats = val_week_df['yhats'].values
                val_week_returns = (val_week_df[val_week_df.position !=0].groupby('date')['returns'].sum().values
                    - tc_per_hour)
                val_week_r_2_pred = 1-np.mean(np.square(val_week_y - val_week_yhats))/np.mean(np.square(val_week_y))
                print(f'\n this week r 2 pred: {val_week_r_2_pred}')
                print(f'this week geom avg ret {QuantTools.calcGeomAvg(val_week_returns)} \n')
        
        # Stop the timer after this hp grid point is completed
        toc = time.perf_counter()

        # For this hp point, add metadata to the results dict
        hps_results_dict['arch_name'] = arch_name
        hps_results_dict['val_start_date'] = val_start_date
        hps_results_dict['val_end_date'] = val_end_date
        hps_results_dict['runtime'] = round((toc - tic)/60, 0) 

        # Add training period statistics
        hps_results_dict['avg_epochs_trained'] = np.mean(num_epochs_trained_list)
        hps_results_dict['avg_num_model_params'] = np.mean(num_model_params_list)
        hps_results_dict['train_r2_pred_min'] = np.min(train_r2_pred_list)
        hps_results_dict['train_r2_pred_mean'] = np.mean(train_r2_pred_list)
        hps_results_dict['train_r2_pred_max'] = np.max(train_r2_pred_list)
        hps_results_dict['train_geom_mean_rtrn_min'] = np.min(train_geom_mean_rtrn_list)
        hps_results_dict['train_geom_mean_rtrn_mean'] = np.mean(train_geom_mean_rtrn_list)
        hps_results_dict['train_geom_mean_rtrn_max'] = np.max(train_geom_mean_rtrn_list)

        # Obtain validation period results
        assert(0 == val_y_yhats_df.isnull().sum().sum()), "Missing observations in the validation period."
        val_y_yhats_df = val_y_yhats_df.sort_values(by=['date', 'asset'], ignore_index=True)
        val_y_yhats_pos_df = QuantTools.formPortfolioPositionsQuantileLongShort(
            val_y_yhats_df, num_prtfl_qntls)
        val_yhats = val_y_yhats_pos_df.yhats.values
        val_ys    = val_y_yhats_pos_df[lhs_col].values
        val_y_yhats_pos_df['returns'] = val_y_yhats_pos_df.position*val_y_yhats_pos_df[lhs_col]
        returns = (val_y_yhats_pos_df.groupby('date')['returns'].sum().values 
            - tc_per_hour)
        assert len(val_yhats) == len(val_ys)

        # Form validation period statistics
        hps_results_dict['val_mse']       = np.mean(np.square(val_ys-val_yhats))
        hps_results_dict['val_r2_pred']   = 1-np.mean(np.square(val_ys-val_yhats))/np.mean(np.square(val_ys))
        hps_results_dict['val_yhat_min']  = np.min(val_yhats)
        hps_results_dict['val_yhat_q1']   = np.quantile(val_yhats, q=0.25)
        hps_results_dict['val_yhat_q2']   = np.quantile(val_yhats, q=0.5)
        hps_results_dict['val_yhat_mean'] = np.mean(val_yhats)
        hps_results_dict['val_yhat_q3']   = np.quantile(val_yhats, q=0.75)
        hps_results_dict['val_yhat_max']  = np.max(val_yhats)
        hps_results_dict['geom_mean_1h']  = QuantTools.calcGeomAvg(returns)
        hps_results_dict['sharpe_annual'] = QuantTools.calcSharpe(returns, periods_in_year=periods_in_year)
        hps_results_dict['sd_annual']     = QuantTools.calcSD(returns, periods_in_year=periods_in_year)
        hps_results_dict['max_dd']        = QuantTools.calcMaxDrawdown(returns)
        hps_results_dict['avg_turnover']  = QuantTools.calcTSAvgTurnover(val_y_yhats_pos_df)

        # Save results to return
        results_list.append(hps_results_dict)

        # For this hp, save results to csv
        cv_df = pd.DataFrame(results_list)
        timestr = time.strftime("%Y%m%d_%H%M%S")
        fp = cv_out_fp + '-' + arch_name + '-' + timestr + '.csv'
        cv_df.to_csv(fp, index=False)
    
    # Return cv results
    return results_list


In [8]:
if __name__ == "__main__":
    # set args
    IN_FP           = '../data/clean/panel_train.pkl'
    ASSET_IN_FP     = '../data/clean/asset_universe_dict.pickle'
    CV_OUT_FP       = '../output/high_dim_fm/cv_results'
    TEST_OUT_FP     = '../data/clean/test_yhats_transformer.pkl'
    LHS_COL         = 'r_ex_tp1'
    VAL_START_DATE  = '2021-01-01'
    VAL_END_DATE    = '2021-01-31'
    TEST_START_DATE = '2021-02-01' # TODO: UP NEXT IS FEB+MARCH TOGETHER
    PERIODS_IN_YEAR = int(365.25*24)
    ARCH_NAME       = 'transfomer'
    LHS_PAD         = -2
    RHS_PAD         = -2
    NUM_PRTFL_QNTLS = 20
    TC_PER_HOUR     = 10.5e-4
    HP_GRID         = {'number_factors': [1],
        'num_lags': [2],                 # NOTE: 3 or 2
        'hidden_dim': [16],              # NOTE: 32, 16, 8
        'num_heads': [2],                # NOTE: 4, 2, 1 (powers of 2)
        'l2_penalty': [1e-4],            # NOTE: 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1
        'dropout_pct': [0.1],            # NOTE: 0.1, 0.2, 0.4
        'initial_learning_rate': [2e-3], # NOTE: 1e-3, 6e-4, 3e-4, 1e-4
        'learning_decay_rate': [0.95],   # NOTE: 0.8, 0.9, 0.95; calc out what itll be across epochs given decay_steps
        'adam_beta_1': [0.9],            # NOTE: 0.95, 0.9, 0.8
        'adam_beta_2': [0.999],          # NOTE: 0.9995, 0.999, 0.99
        'adam_clipnorm': [100],
        'batch_size': [64],
        'num_epochs': [100],
        'early_stopping': [True]}
    
    # read in data
    with open(ASSET_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    all_df = pd.read_pickle(IN_FP)
    
    # subset rows and columns and separate input and output data
    y_df, char_df, ts_df = subsetRowsAndColumns(all_df, LHS_COL)
    gc.collect()

    # pad the lhs data
    y_df = normalizeAndFillMissing(y_df, 
            lhs_col=LHS_COL, lhs_pad=LHS_PAD, rhs_pad=RHS_PAD, 
            ignore_cols=['date', 'asset', LHS_COL])

    # run custom step forward cross validation
    cv_results_list = runCV(y_df, char_df, ts_df, asset_universe_dict, 
        VAL_START_DATE, VAL_END_DATE, TEST_START_DATE,
        LHS_COL, LHS_PAD, RHS_PAD, NUM_PRTFL_QNTLS, TC_PER_HOUR,
        HP_GRID, PERIODS_IN_YEAR, CV_OUT_FP, ARCH_NAME)
